# Тестовое задание  для KazanExpress


![alt text](https://messy-mackerel-9e0.notion.site/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2Faad721cb-b9c6-4cf3-a741-8ac7d119da5f%2Fphoto_2021-09-27_18.40.13.jpeg?table=block&id=a1c88a25-0150-4168-8c12-2b519ecb3078&spaceId=759a960a-baaf-4da2-bac4-158524dd503c&width=250&userId=&cache=v2)

### Установка зависимостей

In [ ]:
!pip install -q pymorphy2[fast]

     |████████████████████████████████| 55 kB 1.8 MB/s 
     |████████████████████████████████| 8.2 MB 7.5 MB/s 
     |████████████████████████████████| 371 kB 36.4 MB/s 


In [ ]:
!pip install -q fasttext

     |████████████████████████████████| 68 kB 2.8 MB/s 


In [ ]:
!pip install -q razdel

In [ ]:
import pandas as pd
import numpy as np
from razdel import tokenize
import sklearn
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
import nltk
from razdel import tokenize, sentenize
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.utils import shuffle
import torchvision.transforms as tt
from sklearn.preprocessing import StandardScaler
import spacy
import fasttext
import pymorphy2
import re
import string
import pickle
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from tqdm.autonotebook import tqdm

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


Загрузка данных

In [ ]:
categories = pd.read_csv('/content/drive/MyDrive/TestKazan/categories_tree.csv', index_col = ['id'])
train = pd.read_parquet('/content/drive/MyDrive/TestKazan/train.parquet')
test = pd.read_parquet('/content/drive/MyDrive/TestKazan/test.parquet')

Отсортируем датасет с категориями и возьмем индексы, чтобы потом осуществить поиск категорий из тренировочного датасета быстрее

In [ ]:
sorted_id = categories.sort_values(by='id').index

Функция для быстрого поиска категории из тренировочного датасета

In [ ]:
def fast_search(number, array, lenght):
  left = 0
  right = lenght -1

  while left <= right:
    median  = (right + left) // 2

    if array[median] == number:
      return True
    elif number > array[median]:
      left = median + 1
    elif number < array[median]:
      right = median - 1

  return False

Определим все имеющиеся листовые категории из трениовочного датасета

In [ ]:
sorted_train = train.sort_values(by='category_id')['category_id'].unique()

В следующем коде формируется словарь соотвествия листовой категории ее пути (родители + сама категория). В связи с тем, что длина пути варьируется, то чтобы уравнять по длине полученные данные, для путей длиной меньше шести незаполненные категории заполняются "-1"

In [ ]:
roots_id = {}

sorted_train = train.sort_values(by='category_id')['category_id'].unique()
lenght = len(train['category_id'].unique())

for id in sorted_id:
  check = fast_search(id,sorted_train,lenght)

  if check:
    parents = []
    while id != 0:
      parents.append(id)
      id = categories.loc[id]['parent_id']
    parents.append(0)
    while len(parents[1:]) < 6:
      parents.append(-1)
    roots_id[parents[0]] = np.array(parents)

Далее, чтобы расширить имеющийся датасет, добавим для каждого товара путь до листовой категории

In [ ]:
paarent_features = []
for id in train['id'].values:
  parents = roots_id[train[train['id'] == id]['category_id'].values[0]]
  paarent_features.append(parents[1:])

In [ ]:
paarent_features = pd.DataFrame(paarent_features, columns = ['parent1','parent2','parent3','parent4','parent5','parent6'])
paarent_features.shape

(283452, 6)

Скопируем исходный тренировочный датасет и объединим его с новыми признаками

In [ ]:
train_df = train.copy()
result = pd.concat([train_df, paarent_features], axis = 1)

В данных присутствуют 2 признака: short_description (краткое описание) и name_value_characteristics (дополнителные характеристики товара). Используем их, чтобы расширить характеристику для каждого товара, сразу заменив отсутствующие значения на "" для удобства конкатенации строк, убрав ненужную пунктуацию для признака name_value_characteristics и создав новый столбец в датафрейме "new_descr". Общая обработка текста будет проведена далее

In [ ]:
result['short_description'].fillna('', inplace = True)
result['name_value_characteristics'].fillna('', inplace = True)


short_desc = []

for value in result['name_value_characteristics'].values:
  if value == '':
    short_desc.append(value)
  else:
    value_div = list(tokenize(value))
    value_div = [x.text for x in value_div if x.text not in string.punctuation]
    short_desc.append(' '.join(value_div))
    
result['new_descr'] = short_desc

Выделим в отдельный список добавим сконкатенированные строки выбранных признаков

In [ ]:
extra_title = [' '.join(x) for x in result[['title','short_description','new_descr']].values]

Далее производится обработка текста. Для обработки были рассмотрены два основных подхода стемминг и лемматизация. Для стемминга блы выбрана библиотека nltk, для лемматизации библиотека pymorphy2. Помимо вышеперечисленных методов, была произведена токенизация (использовалась библиотека razdel, которая отлично справляется с русским языком),все слова были приведены в нижний регистр, убраны все стоп-слова, знаки препинания и выбраны слова длиной более 2. Несмотря на высокие показатели работы библиотек, в процессе работы было сделано интересное наблюдение, ни один из методов практически не обрабатывает слова - приложения (например, красно-белое), и превращает их в одно слово (например, краснобелое), поэтому предварительно, слова такого рода были токенизрованы отдельно.

In [ ]:
nltk.download('stopwords')
stopWords = set(stopwords.words('russian'))
ss = SnowballStemmer(language='russian')
morph = pymorphy2.MorphAnalyzer()

# Обработка слов-приложений
n_titles = [' '.join(str(line).split('-')) for line in extra_title]

# Стемминг
def preproc_stem(text):
    return ' '.join([ss.stem(re.sub(r'[^\w\s]', '', str(word).lower().strip())) for word in word_tokenize(text.lower()) if word not in stopWords and not word in string.punctuation and len(word) > 2])

# Лемматизация
def preproc_lemma(text):
  new_titles = []
  for line in text:
    process_sent = []
    for word in list(tokenize(line.lower())):
      word = word.text
      
      if word not in stopWords and not word in string.punctuation and len(word) > 2:
        
        word = morph.parse(word.lower().strip())

        process_sent.append(word)
        
    new_titles.append(' '.join([res[0].normal_form for res in process_sent]))
  return new_titles

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
new_titles = preproc_lemma(n_titles)

# Основноной метод

В результате работы были проверены множественные гипотезы и апробированы многие модели. Наилучший результат был получен при применении библиотеки **fasttext**. Далее будет рассмотрена работа алгоритма, а также представлены остальные идеи и алгоритмы, показавшие качество хуже

Разделим выборку на тренирочную и тестовую, стратифицируем по категориям товаров

In [ ]:
target = result[['category_id', 'parent1', 'parent2','parent3', 'parent4', 'parent5', 'parent6']]
X_train, X_test, y_train, y_test= train_test_split(new_titles, target, test_size=0.1, random_state=5, stratify = target.category_id.values)

Формируем датасеты для обучения

In [ ]:
df_train = pd.DataFrame({'text':X_train, 'target':y_train['category_id'].values})
df_train['target'] = df_train['target'].apply(lambda x: '__label__' + str(x))
df_train[['target', 'text']].to_csv('/content/train_data.txt', header=False, index=False, sep="\t")


df_test = pd.DataFrame({'text':X_test, 'target':y_test['category_id'].values})
df_test['target'] = df_test['target'].apply(lambda x: '__label__' + str(x))
df_test[['target', 'text']].to_csv('/content/test_data.txt', header=False, index=False, sep="\t")

Для модели подобраны следующие параметры:


*   Использовать биграммы
*   Шаг обучения - 0.6
*   Размерность векторов - 100
*   Число эпох - 6
*   Функция потерь - один против всех





In [ ]:
%%time
model_ft = fasttext.train_supervised(input='/content/train_data.txt', wordNgrams=2, epoch=10, lr = 0.6, dim = 100)

CPU times: user 14min 32s, sys: 953 ms, total: 14min 33s
Wall time: 14min 31s


In [ ]:
model_ft.test('/content/test_data.txt')

(28346, 0.8562054610879842, 0.8562054610879842)

In [ ]:
model_ft.test('/content/test_data.txt')

(28346, 0.8559232343187752, 0.8559232343187752)

Рассчитаем accuracy и добавим предсказания в список

In [ ]:
y_pred = []
a = 0
for x,y in zip(X_test, y_test['category_id'].values):
  p = model_ft.predict(x, k=2)
  pred = int(p[0][0][9:])
  y_pred.append(pred)
  if pred == y:
    a += 1

a / len(X_test)

0.8557115642418683

Функция для расчета иерархического F1-score

In [ ]:
def mode_f1(preds, target):
  ps = [roots_id[pr] for pr in preds]
  ts = target.copy().values

  ps = [np.delete(x,np.where(x == -1))[::-1] for x in ps]
  ts = [np.delete(x,np.where(x == -1))[::-1] for x in ts]
  ps = [np.concatenate([[0,1],x[2:]]) for x in ps]
  ts = [np.concatenate([[0,1],x[2:]]) for x in ts]

  hp = sum(len(set(ts[i]) & (set(ps[i]))) for i in range(len(ts))) / sum(len(set(ps[i])) for i in range(len(ps)))
  hr = sum(len(set(ts[i]) & (set(ps[i]))) for i in range(len(ts))) / sum(len(set(ts[i])) for i in range(len(ts)))

  hf = (2 * hp * hr) / (hp + hr)

  return hf

In [ ]:
f1_score = mode_f1(y_pred, y_test)
f1_score

0.942982719128881

### Таким образом, результат работы **fasttext** на валидационной выборке для иерархического F1-score равен **0.943**

Пример работы fasstext по предсказанию родиельских категорий (описание в README.md)

In [ ]:
preds = {}
accur = {}
for topic in ['parent1', 'parent2','parent3', 'parent4', 'parent5', 'parent6']:
  pred_list = []
  y_test_1 = y_test[topic].values
  y_train_1 = y_train[topic].values

  df_train = pd.DataFrame({'text':X_train, 'target':y_train_1})
  df_train['target'] = df_train['target'].apply(lambda x: '__label__' + str(x))
  df_train[['target', 'text']].to_csv('/content/train_data.txt', header=False, index=False, sep="\t")
  model = fasttext.train_supervised(input='/content/train_data.txt', wordNgrams=2, epoch=10, lr=0.5,loss='ova')
  a = 0
  for x,y in zip(X_test, y_test_1):
    p = model.predict(x, k=1)

    pred = int(float(p[0][0][9:]))
    pred_list.append(pred)
    if pred == y:
      a += 1
  print(topic,' ',a / len(X_test) )
  accur[topic] = a / len(X_test)
  preds[topic] = pred_list

parent1   0.8933535595851266
parent2   0.9157905877372469
parent3   0.9604529739645805
parent4   0.9641572003104495
parent5   0.9645452621181119
parent6   0.991991815423693


Предсказание на тестовом датасете

In [ ]:
test.head()

,id,title,short_description,name_value_characteristics,rating,feedback_quantity,new_descr
0,1070974,Браслет из натуральных камней LOTUS,,,0.000000,0,
1,450413,Fusion Life - Шампунь для сухих и окрашенных в...,,,4.333333,6,
2,126857,"Микрофон для ПК jack 3,5мм всенаправленный","универсальный 3,5 мм микрофон запишет ваш звук",,3.708333,24,
3,1577569,Серьги гвоздики сердце,Серьги гвоздики сердце,,0.000000,0,
4,869328,"Чёрно-красная стильная брошь ""Тюльпаны"" из акр...",Стильная и яркая брошь ручной работы! Великоле...,,0.000000,0,


In [ ]:
test['short_description'].fillna('', inplace = True)
test['name_value_characteristics'].fillna('', inplace = True)


short_desc = []

for value in test['name_value_characteristics'].values:
  if value == '':
    short_desc.append(value)
  else:
    value_div = list(tokenize(value))
    value_div = [x.text for x in value_div if x.text not in string.punctuation]
    short_desc.append(' '.join(value_div))
    
test['new_descr'] = short_desc
test_title = [' '.join(x) for x in test[['title','short_description','new_descr']].values]

In [ ]:
test_title = [' '.join(str(line).split('-')) for line in test_title]

In [ ]:
test_title_pr = preproc_lemma(test_title)

In [ ]:
test_predict = []
for good in test_title_pr:
  p = model_ft.predict(good, k=1)
  pred = int(p[0][0][9:])
  test_predict.append(pred)

In [ ]:
answer = pd.DataFrame(list(zip(test.id.values,test_predict)), columns =['id','predicted_category_id'])
answer.set_index('id')

In [ ]:
answer.to_parquet('/content/drive/MyDrive/TestKazan/result.parquet')

# Другие методы

Мультиномиальный наивный Байес

Несмотря на свою простоту данный метод показывает хорошие результаты в соотношении скорость/качество

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer()

In [ ]:
review_data_tf = cv.fit_transform(new_titles)

In [ ]:
probs = [x/result.shape[0] for x in result.groupby(by = 'category_id').count()['id'].values]

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(review_data_tf, target, test_size=0.1, random_state=5, stratify = target.category_id.values)

In [ ]:
mnb = MultinomialNB(class_prior=probs)

In [ ]:
mnb.fit(X_train, y_train.category_id.values);

In [ ]:
y_pred = mnb.predict(X_test)

In [ ]:
mode_f1(y_pred, y_test)

0.8841337429705888

 Показатель иерархического F1-score для данной модели равено **0.884**

## Линейные методы, построенные на векторном представлении слов

Все линейные модели обучались на построенном векторном представлении, в качестве представления слов была выбрана библиотека **gensim** модуль word2vec. Так как на вход данной модели необходимо подавать не тексты, а отдельные слова, то предварительно разделим наш датасет на слова

In [ ]:
vectors = [list(map(lambda x: x.text,list(tokenize(line)))) for line in new_titles]

In [ ]:
DIM = 50
model = Word2Vec(vectors, size = DIM, window = 3, iter = 15,min_count=1)

Создадим отдельный словарь, куда будем помещать средний вектор всех слов, описывающий товар. Если слова нет в словаре, то добавляем его в отдельный словарь для дальнейшего анализа

In [ ]:
n_f = []
wrong = []
wrong_words = []
for number, line in enumerate(vectors):
  sentence = []
  for word in line:
    if word in model.wv.vocab:
      sentence.append(model[word])
    else:
      wrong_words.append(word)
  if len(sentence) == 0:
    wrong.append(number)
    pass
  else:
    mean_vector = np.mean(sentence, axis=0)

  n_f.append(mean_vector)   

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Были рассмотрены три модели: логистическая регрессия, наивный Байесовский классификатор, метод опорных векторов. Применения решающий деревьев, решающих лесов и ансамблей, построенных на их основе показало свою несостоятельность

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [ ]:
target = result[['category_id', 'parent1', 'parent2','parent3', 'parent4', 'parent5', 'parent6']]
X_train, X_test, y_train, y_test= train_test_split(n_f, target, test_size=0.1, random_state=5, stratify = target.category_id.values)

In [ ]:
%%time
model_1 = LogisticRegression(max_iter = 100)
model_1.fit(X_train, y_train.category_id)
y_pred_lr = model_1.predict(X_test)

In [ ]:
mode_f1(y_pred_lr, y_test)

0.9159046071370748

In [ ]:
model_2 = GaussianNB()
model_2.fit(X_train, y_train.category_id)
y_pred_bn = model_2.predict(X_test)
mode_f1(y_pred_bn, y_test)

0.8669250335956998

In [ ]:
model_3 = SVC(max_iter = 1)
model_3.fit(X_train, y_train.category_id)
y_pred_sv = model_3.predict(X_test)
mode_f1(y_pred_bn, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=5).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


Ни один из этих методов не показал результат выше, чем **fasttext**. **Наивный Байес** показал F1-score равный **0.867**, **логистическая регрессия 0.916** (дальнейшее повышение числа итераций не дает значительного улучшения иерархического F1-score), для **метода опорных векторов** F1-score равен **0.867**

## Применение нейронной сети

Для начала произведем LabelEncoding

In [ ]:
vectors = [list(map(lambda x: x.text,list(tokenize(line)))) for line in new_titles]

In [ ]:
DIM = 50
model = Word2Vec(vectors, size = DIM, window = 3, iter = 15,min_count=1)

In [ ]:
n_f = []
wrong = []
wrong_words = []
for number, line in enumerate(vectors):
  sentence = []
  for word in line:
    if word in model.wv.vocab:
      sentence.append(model[word])
    else:
      wrong_words.append(word)
  if len(sentence) == 0:
    wrong.append(number)
    pass
  else:
    mean_vector = np.mean(sentence, axis=0)

  n_f.append(mean_vector) 

In [ ]:
new_labels = {}
back_labels = {}
n_f = torch.tensor(n_f)
for number, label in enumerate(result.category_id.unique()):
  new_labels[label] = number
  back_labels[number] = label
  
result['neural_labels'] = result['category_id'].map(new_labels)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  This is separate from the ipykernel package so we can avoid doing imports until


Создадим загрузчики данных

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(n_f, torch.tensor(result.neural_labels.values), test_size=0.1, random_state=5, stratify = target.category_id.values)

In [ ]:
dataset_tr = list(zip(X_train, y_train))
dataset_test = list(zip(X_test, y_test))
trainloader = DataLoader(dataset_tr, batch_size = 32, num_workers =2)
testloader = DataLoader(dataset_test, batch_size = 32, num_workers =2)

Для решения задачи была использована следующая нейронная сеть. Написание более глубоких и сложных сетей только ухудшало результат и увеличивало время работы

In [ ]:
class MyNet(nn.Module):
  
  def __init__(self,dim):
    super().__init__()
    self.dim = dim
    self.layers = nn.Sequential(
        nn.Linear(dim, 512),
        nn.BatchNorm1d(1),
        nn.ReLU(inplace = True),
        nn.Linear(512,1024),
        nn.BatchNorm1d(1),
        nn.ReLU(inplace = True),
        nn.Linear(1024,1024),
        nn.Dropout(),
        nn.BatchNorm1d(1),
        nn.ReLU(inplace = True),
        nn.Linear(1024,1231),
        nn.BatchNorm1d(1),
    )

  def forward(self, x):
    
    x = self.layers(x)
    return x

In [ ]:
net = MyNet(DIM)
net = net.cuda()

In [ ]:
def eval_epoch(model, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    processed_size = 0
    best_acc = 0.0


    for inputs, labels in val_loader:
        inputs = inputs.to(DEVICE).unsqueeze(1)
        labels = labels.to(DEVICE)

        with torch.set_grad_enabled(False):
            outputs = model(inputs).squeeze(1)
            loss = criterion(outputs, labels)
            preds = torch.argmax(outputs, 1)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_size += inputs.size(0)
    
    val_loss = running_loss / processed_size
    val_acc = running_corrects.double() / processed_size
    #сохраняем сеть с наилучшими показателями
    if val_acc > best_acc:
      best_acc = val_acc
      best_model_wts = model.state_dict()
      torch.save(model.state_dict(),'/content/drive/MyDrive/TestKazan/net.w')
    
    return  val_loss, val_acc

In [ ]:
def fit_epoch(model, train_loader, criterion, optimizer,scheduler):
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0
    

    for inputs, labels in tqdm(train_loader):
        inputs = inputs.to(DEVICE).unsqueeze(1)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(inputs).squeeze(1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        preds = torch.argmax(outputs, 1)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_data += inputs.size(0)
    scheduler.step()
    train_loss = running_loss / processed_data
    train_acc = running_corrects.cpu().numpy() / processed_data


    return train_loss, train_acc

In [ ]:
def train(train_loader,val_loader, model, epochs):

    
    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
    val_loss {v_loss:0.4f} train_acc {t_acc:0.4f} val_acc {v_acc:0.4f}"
    
    with tqdm(desc="epoch", total=epochs) as pbar_outer:


        opt = torch.optim.Adam(model.parameters(),lr = 3e-4, betas = (0.5,0.99))

        scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=10, gamma=1e-1)

        #criterion is Cross entropy loss
        criterion = nn.CrossEntropyLoss()

        for epoch in range(epochs):
            train_loss, train_acc = fit_epoch(model, train_loader, criterion, opt,scheduler)
            print("loss", train_loss)
            
            val_loss, val_acc = eval_epoch(model, val_loader, criterion)
            history.append((train_loss, train_acc, val_loss, val_acc))
            
            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch+1, t_loss=train_loss,\
                                           v_loss=val_loss, t_acc=train_acc, v_acc=val_acc))

    return history

In [ ]:
DEVICE = torch.device('cuda')
data = train(trainloader, testloader, net, 25)

epoch:   0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/7530 [00:00<?, ?it/s]

loss 1.48545204728135

Epoch 001 train_loss: 1.4855     val_loss 1.0539 train_acc 0.6804 val_acc 0.7513


  0%|          | 0/7530 [00:00<?, ?it/s]

loss 0.8816383763572205

Epoch 002 train_loss: 0.8816     val_loss 0.8592 train_acc 0.7808 val_acc 0.7875


...

loss 0.1987766294739786

Epoch 023 train_loss: 0.1988     val_loss 1.6138 train_acc 0.9368 val_acc 0.8181


  0%|          | 0/7530 [00:00<?, ?it/s]

loss 0.19647394888656325

Epoch 024 train_loss: 0.1965     val_loss 1.6218 train_acc 0.9377 val_acc 0.8180


  0%|          | 0/7530 [00:00<?, ?it/s]

loss 0.19458560007283898

Epoch 025 train_loss: 0.1946     val_loss 1.6304 train_acc 0.9384 val_acc 0.8175


In [ ]:
preds_list = []
for inputs, labels in testloader:
  inputs = inputs.to(DEVICE).unsqueeze(1)
  labels = labels.to(DEVICE)

  with torch.set_grad_enabled(False):
      outputs =net(inputs).squeeze(1)
      preds = torch.argmax(outputs, 1)
  preds_list.extend(preds.cpu().numpy())

In [ ]:
real_preds = [back_labels[x] for x in preds_list]

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(n_f, target, test_size=0.1, random_state=5, stratify = target.category_id.values)

In [ ]:
f1_score = mode_f1(real_preds, y_test)
f1_score

0.9282490776309005

### Использование нейронной сети также не позволило получить результат выше, чем для **fasttext**. Максимальное значения для иерархического **F1-score 0.928**

# TF-IDF и логистическая регрессия

Обучим векторайзер и поулчим веткорные предствления слов в подходе TF-IDF

In [ ]:
vectorizer = TfidfVectorizer(ngram_range = (1,2), min_df = 0.01, max_df=0.8)
vectors = vectorizer.fit_transform(new_titles)
dense_vectors = vectors.todense()

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(dense_vectors, target, test_size=0.2, random_state=0)
y_train.shape, y_test.shape

((226761, 7), (56691, 7))

In [ ]:
model = LogisticRegression(max_iter=20)
model.fit(X_train, y_train.category_id.values)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(max_iter=20)

In [ ]:
y_pred = model.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


In [ ]:
f1_score = mode_f1(y_pred , y_test)
f1_score

0.6714014557084995

**TF-IDF векторайзинг и логистическая регрессия** показали наименьший результат на иеерархическом F1-score **0.67**